# Test Notebook for nisarcryodb
---

This note book is used to develop, test, and document the  `nisarcryodb` class, which is used to query the NISAR cal/val db for GPS data from cryosphere validation sites.


In [1]:
%load_ext autoreload
%autoreload 2
import nisarcryodb
from psycopg2 import sql
import matplotlib.pyplot as plt
import numpy as np

## Initialize nisarcryodb Object

This step initializes the connection to the nisar cal/val data base. It requires a cal/val database user name and passwd.

In [2]:
myConnection = nisarcryodb.nisarcryodb()

User name:  irj
Password:  ········


## List Schema

List all of the schema in the database. 

In [3]:
schemas = myConnection.listSchema(quiet=False)

pg_catalog
information_schema
soil_moisture
pg_temp_7
pg_toast_temp_7
pg_temp_8
pg_toast_temp_8
disturbance
corner_reflectors
landice
solidearth
crop_area
inundation
pg_temp_16
pg_toast_temp_16
biomass
seaice
nisar
permafrost
tiger
tiger_data
topology
public


## List Table Names for Schema

List the names of the table under the landice schema.

In [4]:
myConnection.listSchemaTableNames('landice');

site
requirement
gps_data
gps_data_version
l3_product
gps_test_data
l3_product_calval_site_jct
l3_product_requirement_jct
validation_run
validation_run_gps_data_jct
velocity_data
gps_station


## List Column Names for Table

List the column names for a table (e.g., gps_station) and schema (e.g., landice).

In [5]:
stationColumns = myConnection.listTableColumns('landice', 'gps_station', quiet=False)

station_id, station_name, reflat, reflon, refheight, poc_name, poc_email, station_geom, site_id


This time get the columns for the gps_data table. Use `returnType=True` to get the data type for each column.

In [6]:
gpsColumns, gpsDataTypes = myConnection.listTableColumns('landice', 'gps_data', returnType=True)
gpsColumns

gps_data_id, station_id, version_id, date_uploaded, measurement_dt_utc, decimal_year, nominal_doy, lat, lon, ht_abv_eps, sigma_e, sigma_n, sigma_v, tides_x, tides_y, tides_z, data_geom
integer, integer, integer, date, timestamp without time zone, double precision, integer, double precision, double precision, double precision, real, real, real, real, real, real, USER-DEFINED


['gps_data_id',
 'station_id',
 'version_id',
 'date_uploaded',
 'measurement_dt_utc',
 'decimal_year',
 'nominal_doy',
 'lat',
 'lon',
 'ht_abv_eps',
 'sigma_e',
 'sigma_n',
 'sigma_v',
 'tides_x',
 'tides_y',
 'tides_z',
 'data_geom']

## List Column Values

Get the `station_name` column values for `landice.gps_station` tables. Select only those that begin with "N".

In [7]:
stationNames = myConnection.getColumn('landice', 'gps_station', 'station_name')
[station for station in stationNames if station.startswith("N")]

['NIU1',
 'NIU2',
 'NIU3',
 'NIU4',
 'NIU5',
 'NIT0',
 'NIT1',
 'NIT2',
 'NIT3',
 'NIT4',
 'NIT5',
 'NIL1',
 'NIL2',
 'NIL3',
 'NIL4',
 'NIL5',
 'NIT7']

Get the `latitude` column values for `landice.gps_data` tables 

In [8]:
lat = myConnection.getColumn('landice', 'gps_data', 'lat');
lat[0:10]

[68.97840725,
 68.97840724,
 68.9784072,
 68.97840724,
 68.97840721,
 68.97840721,
 68.97840722,
 68.97840723,
 68.97840729,
 68.97840723]

## Station Name to ID

Determine the station ID (e.g., 22) from the station name (e.g., LORG).

In [9]:
myConnection.stationNameToID('NIL3')

9

## Get GPS Data for Date Range

Return all the data for a designated station for the decimal date range (d1, d2). 

In [20]:
d1 = 2024.8165544457129-1
d2 = 2025.819178050482+1
myData = myConnection.getStationDateRangeData('NIL3', d1, d2, schemaName='landice', tableName='gps_data')

SELECT * FROM landice.gps_data WHERE decimal_year BETWEEN %(val1)s AND %(val2)s AND station_id = %(station_id)s ;


In [17]:
plt.plot(myData['lon'], myData['lat'], 'r.')
plt.xlabel('longitude')
plt.ylabel('latitude')

TypeError: 'NoneType' object is not subscriptable

## Get Station Information

Return the table with all of the top-level station data (e.g,, name, id, position, etc).

In [ ]:
myConnection.getTableListing(schemaName='landice', tableName='gps_station')

## Get Level 3 Products

Get all L3 products from 2015 to 2023. Filters can be specified by column heading and value (e.g., `filters={'measurement_type': 'velocity'}`. Wild can be set off with % signs. For example, to list only the velocity files with 'vx' in the name will be accomplished by `filters={'product_path': '%vx%'}`.

In [ ]:
date1 = '01-01-2015'
date2 = '12-31-2023'
myConnection.getL3DateRangeData(date1, date2, schemaName='landice', tableName='l3_product', filters={'product_path': '%vx%', 'measurement_type': 'velocity'})

## Rollback a Query Error

This will reset the connection after a query fails with an error. Most of the routines should trap errors and automatically rollback.

In [ ]:
myConnection.connection.rollback()

## Close the connection

Close the connection when finished.

In [ ]:
myConnection.close()